In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import datasets, transforms
from torchvision.utils import save_image
import time
import matplotlib.pyplot as plt
import numpy as np
from torchviz import make_dot
from torchinfo import summary
# Define constants
IMG_SIZE = 128  # Set to 128x128 for the new output size
LATENT_DIM = 200  # Updated latent dimension
BATCH_SIZE = 100  # Updated batch size
EPOCHS = 100
epoch_times = []  # List to store time taken for each epoch
d_loss_values = []
g_loss_values = []
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(100, 2048, 4, 1, 0, bias=False),
            nn.BatchNorm2d(2048),
            nn.ReLU(True),
            nn.ConvTranspose2d(2048, 1024, 4, 2, 1, bias=False),
            nn.BatchNorm2d(1024),
            nn.ReLU(True),
            nn.ConvTranspose2d(1024, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 3, 4, 2, 1, bias=False),
            nn.Tanh()

        )

    def forward(self, input):
        return self.main(input)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Flatten(),
            nn.Linear(512 * 8 * 8, 1),  # Adjusted for 128x128 images
            nn.Sigmoid()
        )

    def forward(self, img):
        validity = self.model(img)
        return validity

transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

dataloader = DataLoader(
    ConcatDataset([datasets.Flowers102(root='../../../../data/flowers', split='train', download=True, transform=transform),
                   datasets.Flowers102(root='../../../../data/flowers', split='val', download=True, transform=transform),
                   datasets.Flowers102(root='../../../../data/flowers', split='test', download=True, transform=transform)]),
    batch_size=BATCH_SIZE, shuffle=True
)

# Initialize models
generator = Generator().to(device)
discriminator = Discriminator().to(device)
# Generate and save summary for Generator model
generator_summary = summary(generator, input_size=(1, LATENT_DIM), verbose=2, col_names=["input_size", "output_size", "num_params", "kernel_size", "mult_adds"])
with open("generator_summary_128.txt", "w", encoding="utf-8") as f:
    f.write(str(generator_summary))

# Generate and save summary for Discriminator model
discriminator_summary = summary(discriminator, input_size=(1, 3, IMG_SIZE, IMG_SIZE), verbose=2, col_names=["input_size", "output_size", "num_params", "kernel_size", "mult_adds"])
with open("discriminator_summary_128.txt", "w", encoding="utf-8") as f:
    f.write(str(discriminator_summary))

# Visualize the Generator Model
z_sample = torch.randn(1, LATENT_DIM).to(device)  # A single sample input for the generator
gen_sample_output = generator(z_sample)
gen_dot = make_dot(gen_sample_output, params=dict(generator.named_parameters()))
gen_dot.render("generator_architecture_128", format="png")  # Save the visualization as a PNG file

# Visualize the Discriminator Model
real_img_sample = torch.randn(1, 3, IMG_SIZE, IMG_SIZE).to(device)  # A single sample input for the discriminator
disc_sample_output = discriminator(real_img_sample)
disc_dot = make_dot(disc_sample_output, params=dict(discriminator.named_parameters()))
disc_dot.render("discriminator_architecture_128", format="png")  # Save the visualization as a PNG file
# Loss function
adversarial_loss = nn.BCELoss()

# Optimizers
optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Training
for epoch in range(EPOCHS):
    epoch_start_time = time.time()  # Start time for the epoch
    for i, (imgs, _) in enumerate(dataloader):

        # Configure input
        real_imgs = imgs.to(device)
        batch_size = real_imgs.size(0)
        # Calculate and print GPU memory usage
        allocated_memory = torch.cuda.memory_allocated() / (1024 ** 3)  # Convert to GB
        reserved_memory = torch.cuda.memory_reserved() / (1024 ** 3)    # Convert to GB
        # Adversarial ground truths
        valid = torch.ones(batch_size, 1, requires_grad=False).to(device)
        fake = torch.zeros(batch_size, 1, requires_grad=False).to(device)

        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = torch.randn(batch_size, LATENT_DIM).to(device)

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss for real images
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        # Loss for fake images
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        # Total discriminator loss
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        # -----------------
        #  Train Generator
        # -----------------
        optimizer_G.zero_grad()

        # Loss for fake images with flipped labels
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        print(f"[Epoch {epoch}/{EPOCHS}] [Batch {i}/{len(dataloader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")
        print(f"[GPU Memory Allocated: {allocated_memory:.2f} GB] [GPU Memory Reserved: {reserved_memory:.2f} GB]")

    # Save sample images
    if epoch % 2 == 0:
        save_image(gen_imgs.data[:25], f"images/{epoch}_DCGAN_flowers2_128.png", nrow=5, normalize=True)
        # Save the model
        torch.save(generator.state_dict(), f"saved_model_dcgan_flower2_128_{epoch}.pth")
        d_loss_values.append(d_loss.item())
        g_loss_values.append(g_loss.item())
    epoch_end_time = time.time()  # End time for the epoch
    epoch_duration = epoch_end_time - epoch_start_time  # Duration of the epoch
    epoch_times.append(epoch_duration)  # Append the duration to the list
# Save model after last iter
save_image(gen_imgs.data[:25], f"images/{epoch}_DCGAN_flowers2_128.png", nrow=5, normalize=True)
# Save the model
torch.save(generator.state_dict(), f"saved_model_dcgan_flower2_128_{EPOCHS}.pth")
# Calculate average time per epoch
average_time_per_epoch = sum(epoch_times) / len(epoch_times)
print(f"Average time per epoch: {average_time_per_epoch:.2f} seconds")
plt.plot(np.arange(0, EPOCHS,2), d_loss_values, label='Discriminator loss')
plt.plot(np.arange(0, EPOCHS,2), g_loss_values, label='Generator loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss values')
plt.savefig('loss_values_flowers_128.png')
plt.show()
# Create plot for time taken per epoch
plt.plot(np.arange(0, EPOCHS), epoch_times)
plt.title("Time taken per epoch")
plt.xlabel("Epoch")
plt.ylabel("Time (s)")
plt.savefig("time_per_epoch_flowers_128.png")
plt.show()